# How are Taxis enhancing the Singapore's Public Transport System.

## Introduction
Bus and rail system have fixed routes and schedules. Therefore they have deterministic coverage patterns which meet the general transportation demand. Taxicabs and ride sharing services do not have fixed routes or schedules thus in some sense, they meet the "ad hoc" demands. The goal of the project is to see if the supply pattern of taxicabs fill the gaps in the public transport network thereby enhancing it. 
The study would be done for Singapore, a city state which provides plentiful data for the public transport network. While public transport modelling has been done before, to the best of our knowledge, there have not been an analysis of taxis and public transport together.

## Existing Work

1. Singapore in Motion: Insights on Public Transport Service Level Through Farecard and Mobile Data Analytics, IBM 2016
http://www.kdd.org/kdd2016/papers/files/SingaporeInMotion_v3.pdf
2. Time-Series Data Mining in Transportation: A Case Study on Singapore Public Train Commuter Travel Patterns, SMU 2014 

We got inspired from two other projects. The projects are developed and made open source for commuters to use. They are.
   
   
   1. [Taxi Router SG](https://github.com/cheeaun/taxirouter-sg) by [Lim Chee Aun](https://twitter.com/cheeaun). Its idea was to showcase the following details
       
       * Taxi stands in Singapore.
       * Shows all available taxis in the whole Singapore.
       * How many available taxis around the commuter?
       * How far is the nearest taxi stand around the commuter?

   2. [TaxiSg](http://uzyn.github.io/taxisg/) by [U-Zyn Chua](https://twitter.com/uzyn). This app helps the commuter to understand the distribution of taxis during a historic window (ranging from 15 minutes to 2 weeks of historic data).
   
   
   Both the apps get their data from a government organisation called, The Land Transport Authority (LTA) of Singapore. LTA publishes a wide variety of transport-related datasets (static and dynamic / realtime) on their DataMall platform for enterprises, third-party developers, and other members of the public to promote citizen co-creation of innovative and inclusive transport solutions. 

## Problem Statement

   We wanted to answer the following questions and provide inferences based on our results.
   
   1. Is there a difference between the density distribution of taxis (AdHoc Requests made by commuters) and Bus/Rail network (Planned Transportation network) over the period of time?
   
   2. Are the taxis really trying to fill the gaps of the public transport system? (Fully loaded buses)
   
   3. Is there any change in the distribution of taxis vs Public transport system between weekdays and weekends? Statistically decide on a location (identify top ten location) and KL divergence on Relative frequencies of bus vs taxi during the window/ for the whole day
   
   4. Can the Taxi demand be predicted based on previous patterns observed from the Taxi/Bus Density distribution? How precise can the prediction be in terms of time/location.
   Try [Auto Regression](http://machinelearningmastery.com/autoregression-models-time-series-forecasting-python/) 
  
   

## Data Sets
2 dynamic data sets are collected were collected using the API between 03/15/2017 to 03/19/2017.

1. Taxi Availability 
2. Bus Arrival

2 static data was collected
1. Urban Land Authority Master Planning Sub Zone 2014
2. MRT train schedule (work in progress)

The URA Zone codes were use to determine the usage of land, whether is it for commercial or residential etc.
<img src="images/ura_2014.png" width='500pix'>

## Taxi Availability Dataset

### Description
Returns location coordinates of all Taxis that are currently available for hire. Does not include "Hired" or "Busy" Taxis. We polled the API every *1min* for this dataset. A total of **40982444** location records were collected.
                                                                                                             
| **Attributes** 	| **Description**                                                    	|
|----------------	|--------------------------------------------------------------------	|
| Latitude       	| provides the latitude of the location where the taxi is available  	| 
| Longitude      	| provides the longitude of the location where the taxi is available 	| 
| Date           	| provides the date when the taxi was available                      	| 
| Time           	| provides when the time was available                               	| 



## Bus Arrival Dataset

### Description
Returns real-time Bus Arrival information for Bus Services at a queried Bus Stop, including: Estimated Time of Arrival (ETA), Estimated Location, Load info. We polled the API all over the bus stops in Singapore every *6min* for this dataset. A total of **6394212** bus stop arrival records were collected.

                                                                                                                 
| **Attributes**    | **Description**                                                       |
|----------------   |--------------------------------------------------------------------   |
| ServiceNo         | Bus service number   | 
| Status         | Bus Status    | 
| Latitude | Estimated location coordinates of bus |
| Longtitude | Estimated location coordinates of bus |
|Load|  Bus occupancy / crowding: Seats Available, Standing Available, Limited Standing|


## Data Collection
[hui han]
### LTA Datamall

### Dates of Collection
in SG time
14/3/17 - 19/4/17

## Modelling transportation flow

### Projection from Lat Lon to UTM [hui han] 
   We wanted to change the projection from the regular (Latitude,Longitude) to Universal Transverse Mercator(UTM) for our project. You can find out the basics of  UTM [here](http://gisgeography.com/utm-universal-transverse-mercator-projection/). The primary reason for our decision is to avoid representing the Geo Locations in a distorted manner. UTM is also the best format to project for narrow Geo-locational area with high density details. 

### Density Estimation
       
   We had data collected for a five day period. To answer all the questions in our problem statement, we had to start with a Kernel Density estimation of Taxis and Buses for a window (15 minute window) slided over a period of five days. Kernel Desity Estimation will help us to identify the latent distribution from which the Taxi and Bus data originate. Also, Knowledge of the Distribution would help us to predict the taxis in the future based on the historic allocation of taxis and buses. We initially collected a random 10 minute sample of Bus Data and Taxi Data (for the same dates) and plotted Distribution of the Random sample. Before getting into the design decisions to formulate algorithm, we identified an outlier in the dataset.
   

### Outlier Removal 

  The Changi Airport is one of the biggest Taxi hub in Singapore. At the same time, there are not many buses which are going to this destination. Our reasoning is that, commuters use train to travel to the airport. So if we compare the distribution between taxis and buses and if we try to answer the questions in the problem statement, our answers in the airport location would be biased. Hence we decided to remove Taxis from the Airport area location. When we remove the Airports area from equation, we see many hotspot locations in the sampled dataset.
  
  images go here.


### Different Kernel Density Estimation

  
  Here is a primer to [Kernel Density Esitmation](https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/). We chose KDE because:
Its non-parametric, which means that it attempt to estimate the density directly from the data without assuming a particular form for the underlying distribution. We cannot assume real-life events to happend based on certain distribution. We do have problems with Kernel Density Estimation.

   1. What Bandwidth or window to use?
   2. What Kernel function to use to count the number of Spatial points that fall in the vicinity of the proposed bandwidth.
   
   The below infographic will help to understand the Factors to consider while designing KDE
   
  <a href="http://research.cs.tamu.edu/prism/lectures/pr/pr_l7.pdf"> <img src="images/KernelDensityInfographic.png"  width="700px"></a>
  

  
  The Infographic shows the most simplest of all the kernel functions which is there. But Parzen Kernals weight every point from the center in the same way. It yields Density Estimates that have discontinuities. So we had to choose a Kernel which smoothen the effects of the above drawbacks. Though there are many [Kernels](https://goo.gl/fCN79N), We decided to go with the Epanechnikov which is the most common Kernel that is put in use. 
  
  Now we have decided on a Kernel, We have to decide on a free parameter, the Bandwidth. The problems with deciding on Bandwidth are that, if we increase the bandwidth (a radius for spatial area) We might oversmooth the Kernel density that we are trying to estimate. If we use a small bandwidth, we might undersmooth, i.e, we might get muliple normal distribution peaks in the spatial graph. We will have to find a h that will reduce the error between estimated density and the true density. But we don't know the true density of the spatial data that we have and its altogether a different problem to solve. So we decided to go with a kernel bandwidth used in spatial analyis tool called QGIS.
  
  
  
  
  


### Generating large scale KDE 



## Inference
### Comparison of Transport Densities

## Demand Estimation
### Prediction Model 
[hui han]

## Recommendation

## References

## Code Appendix

## Taxi API Code

In [6]:
#!/home/bks4line/anaconda2/bin/python
# Author : Karthik Balasubramanian

import json
import urllib
from urlparse import urlparse
import httplib2 as http #External library
import pandas as pd
import time
from datetime import datetime
from pytz import timezone
import os
#  please get your account keys and place here
headers = { 'AccountKey' : 'XXXXX','accept' : 'application/json'}

uri = 'http://datamall2.mytransport.sg/' #Resource URL
path = 'ltaodataservice/Taxi-Availability?$skip='
fmt =  '%Y-%m-%d_%H:%M:%S'
sg = timezone('Asia/Singapore')
my_path = %pwd
dir_path = my_path+"/data"



def get_data_from_LTA(filename):
    
    global headers,uri,path,fmt,sg,dir_path

    
    #Build query string & specify type of API call
    
    final_list = []
    target = urlparse(uri + path+str(len(final_list)))

    
    
    method = 'GET'
    body = ''

    #Get handle to http
    h = http.Http()
    
    # Obtain results
    response, content = h.request(target.geturl(),method,body,headers)

    # Parse JSON to print
    jsonObj = json.loads(content)
    
    final_list.extend(jsonObj["value"])
    
    while(len(jsonObj["value"])>0):
        target = urlparse(uri + path+str(len(final_list)))
        # print target.geturl()
        response, content = h.request(target.geturl(),method,body,headers)
        jsonObj = json.loads(content)
        final_list.extend(jsonObj["value"])
    
    
    time_now_in_sg = datetime.now(sg)
    date_and_time_ff =  time_now_in_sg.strftime(fmt)
    date_and_time = date_and_time_ff.split("_")
    date_in_sg = [date_and_time[0]]*len(final_list)
    time_in_sg =  [date_and_time[1]]*len(final_list)
    
    df = pd.DataFrame(final_list)
    df['date'] = pd.Series(date_in_sg, index=df.index)
    df['time'] = pd.Series(time_in_sg, index=df.index)
    
    if not filename:
        filename =  dir_path+"/taxi_"+date_and_time_ff+".csv"
        df.to_csv(filename)
    else:
        file_size_exceed = float(os.path.getsize(filename))/float(5e+6)
        if file_size_exceed>1.0:
            print "file_size_exceed"
            filename = dir_path+"/taxi_"+date_and_time_ff+".csv"
            print "new file name {0}".format(filename)
            df.to_csv(filename)
        else:
            print "file size not exceeded"
            df.to_csv(filename, mode='a', header=False)

    return filename


#  run the below code 

 
# starttime =  time.time()
# filename = None
# # get_data_from_LTA(filename=None)
# while True:
#     filename = get_data_from_LTA(filename)
#     starttime =  time.mktime(datetime.now().timetuple())
#     time.sleep(50.0 - ((time.time() - starttime) % 60.0))

